# 0.1 imports

In [21]:
import pandas as pd
import numpy as np

from statsmodels.tsa.api import SimpleExpSmoothing

# 0.2 helpers functions

# 0.3 load data

In [22]:
path = '/home/lavino/Documentos/projetos_indicium/ds_desafio/'

In [23]:
df_raw = pd.read_csv(path + 'data/df_fill.csv', index_col='year', parse_dates=True)
df_paramns = pd.read_csv(path + 'data/df_countrys_paramns.csv')

In [24]:
df_raw = df_raw.asfreq(freq='AS')

# 1.0 Predictions

In [25]:
df1 = df_raw.copy()

## 1.1 train test split

In [26]:
test_years = 5
train = df1[:-test_years]
test = df1[-test_years:]

## 1.2 train and predict for individual country

In [27]:
predict = pd.DataFrame()
for country in df1.columns:
    paramns = eval(df_paramns[df_paramns['country'] == country]['best_params'].values[0])
    model = SimpleExpSmoothing(
        train[country], 
        initialization_method=paramns['initialization_method']
    ).fit(
        smoothing_level=paramns['smoothing_level'], 
        optimized=False
    )
    
    forecast = model.forecast(len(test))
    
    predict = pd.concat([predict, pd.DataFrame(forecast, columns=[country])], axis=1)

## 1.3 Save dataframe

In [28]:
predict.to_csv(path + 'data/predict.csv', index=True)